## EC22181 - APAMPA MUTIAH

#### AI Coursework1

#### Task 2.1

In [1]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
from queue import PriorityQueue, Queue
import numpy as np

In [2]:
df = pd.read_csv("tubedata.csv", header=None)
df.head()

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


In [3]:
station_dict = defaultdict(list)
zone_dict = defaultdict(set)
# get data row by row
for index, row in df.iterrows():
    start_station = row[0]
    end_station = row[1]
    act_cost = int(row[3])
    line = row[2]
    zone1 = str(row[4])
    zone2 = str(row[5])
  
    station_list = station_dict[start_station]
    station_list.append((end_station, act_cost, line))
    
    station_list = station_dict[end_station]
    station_list.append((start_station, act_cost, line))
    
 
    zone_dict[start_station].add(zone1)

    if zone2 != "0":
        zone_dict[end_station].add(zone2)
    else:
        zone_dict[end_station].add(zone1)
    
    # To give the zones with multiple zones only the main zone for heuristic function 
    if len(zone_dict[start_station]) > 1:
        zone_dict[start_station].clear()
        zone_dict[start_station].add(zone1)

In [4]:
def convert_to_dict(dict_t):
    station_dict1 = dict()
    for x, y in dict_t.items():
        a_dict = dict()
        for a, b, c in y:
            a_dict[a] = {"weight" : b, "line" : c}
        station_dict1[x] = a_dict
    
    return station_dict1

In [5]:
station_dict1 = convert_to_dict(station_dict)
t_graph = nx.Graph(station_dict1)

In [6]:
def weighted_graph(graph, node_size, font_size, fig_size):
    plt.figure(num=None, figsize=fig_size, dpi=100)
    plt.axis('off')
  
    nodes_point = nx.spring_layout(graph) 
  
    edge_w  = nx.get_edge_attributes(graph,'weight')
    
    nx.draw_networkx_nodes(graph, nodes_point, node_size=node_size, node_color = ["cyan"]*graph.number_of_nodes())

    nx.draw_networkx_edges(graph, nodes_point, edgelist=list(graph.edges), width=3)
    nx.draw_networkx_edge_labels(graph, nodes_point, edge_labels = edge_w)

    nx.draw_networkx_labels(graph, nodes_point, font_size=font_size)
    
    plt.axis('off')
    plt.show()
weighted_graph(t_graph, 1500, 20, (100,100))

In [7]:
# To implement BFS
def bfs_implementation(train_dict, start_st, end_st, compute_exploration_cost=False, reverse=False):
    if start_st == end_st:
        return None
      
    number_of_explored_nodes = 0
    visited_node = {start_st}
    bfs_queue = Queue()
    bfs_queue.put(start_st)
    
    path = []
    cost = 0
    
    parent = {}
    parent[start_st] = None
    
    while bfs_queue:
        node = bfs_queue.get() # pop from the right of the list
        number_of_explored_nodes += 1
        
        if node == end_st:
            path = [node]
            while parent[node]:
                node = parent[node]
                path = [node] + path
            if compute_exploration_cost:
                #print('number of explorations = {}'.format(number_of_explored_nodes))
                for i in range(len(path) - 1):
                    cost += train_dict[path[i]][path[i + 1]]['weight']
                #print('Cost in average time = {} minutes'.format(cost))
            return path, number_of_explored_nodes, cost
        
        neighbours = reversed(train_dict[node]) if reverse else train_dict[node]
        
        for neighbour in neighbours:
            if neighbour not in visited_node:
                bfs_queue.put(neighbour)
                visited_node.add(neighbour)
                parent[neighbour] = node
    return None

# To implement DFS
def dfs_implementation(train_dict, start_st, end_st, compute_exploration_cost=False, reverse=False):
    if start_st == end_st:
        return None
      
    number_of_explored_nodes = 0
    visited_node = {start_st}
    bfs_queue = [start_st]
    
    path = []
    cost = 0
    
    parent = {}
    parent[start_st] = None
    
    while bfs_queue:
        node = bfs_queue.pop() # pop from the right of the list
        number_of_explored_nodes += 1
        
        if node == end_st:
            path = [node]
            while parent[node]:
                node = parent[node]
                path = [node] + path
            
            if compute_exploration_cost:
                #print('number of explorations = {}'.format(number_of_explored_nodes))
                for i in range(len(path) - 1):
                    cost += train_dict[path[i]][path[i + 1]]['weight']
                #print('Cost in average time = {} minutes'.format(cost))
            return path, number_of_explored_nodes, cost
        
        neighbours = reversed(list(train_dict[node])) if reverse else list(train_dict[node])
        
        for neighbour in neighbours:
            if neighbour not in visited_node:
                bfs_queue.append(neighbour)
                visited_node.add(neighbour)
                parent[neighbour] = node
    return None

# To implement UCS

def ucs_implementation(graph, start_st, end_st):
    
    if start_st == end_st:
        return None
    
    number_of_explored_nodes = 0
    cost = 0
    explored = {}
    explored[start_st] = (cost, start_st)

    # Create priority queue
    p_queue = PriorityQueue()
    p_queue.put((0, [start_st]))
    
    while p_queue.empty() == False:
    
        cost, nodes = p_queue.get() 
    
        node = nodes[-1]
        number_of_explored_nodes += 1
        
        if node == end_st:
            path = explored[end_st][1]
            cost = explored[end_st][0]
            return path, number_of_explored_nodes, cost
            
            #print('number of explorations = {}'.format(number_of_explored_nodes))
            #print("The cost in average time is {} minutes".format(explored[end_st][0]))
            #print("Path is {}".format(explored[end_st][1]))
        
        neighbours = graph.neighbors(node)

        for neighbour in neighbours:
            # Get the cost of each node
            edge_weight = graph[node][neighbour]
            if len(edge_weight) != 0:
                cost_to_neighbour = edge_weight["weight"]
            else:
                cost_to_neighbour = 1
                
            new_cost = cost + cost_to_neighbour
            
            if (neighbour not in explored) or (explored[neighbour][0]>new_cost):
                next_node = (new_cost, nodes+[neighbour])
                explored[neighbour] = next_node # Update the node with best value
                p_queue.put(next_node) # Also will add it as a possible path to explore
            
    
    return None

In [8]:
bfs_implementation(station_dict1, 'Euston', 'Victoria', compute_exploration_cost=True)

(['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'], 35, 7)

In [9]:
dfs_implementation(t_graph, 'Euston', 'Victoria', compute_exploration_cost=True)

(['Euston',
  'Warren Street',
  'Goodge Street',
  'Tottenham Court Road',
  'Leicester Square',
  'Charing Cross',
  'Embankment',
  'Westminster',
  'Green Park',
  'Victoria'],
 136,
 15)

#### Task 2.2

In [10]:
data = [[], [], [], [], [], []]
cities = [['Euston', 'Victoria'],
          ['Canada Water', 'Stratford'],
          ['New Cross Gate', 'Stepney Green'],
          ['Ealing Broadway', 'South Kensington'],
          ['Mile End', 'Baker Street'],
          ['Baker Street', 'Wembley Park']]
col_names = ['Train stations', 'Route', 'Exploration Number', 'Cost in minutes']
for i, x in enumerate(cities):
    p, n, c = bfs_implementation(t_graph, x[0], x[1], compute_exploration_cost=True)
    data[i].append(cities[i])
    data[i].append(p)
    data[i].append(n)
    data[i].append(c)

In [11]:
# Data for BFS
df = pd.DataFrame(data, columns = col_names)
df.style.set_properties(subset=['Route'], **{'width': '5000px'})
df

,Train stations,Route,Exploration Number,Cost in minutes
0,"[Euston, Victoria]","[Euston, Warren Street, Oxford Circus, Green P...",40,7
1,"[Canada Water, Stratford]","[Canada Water, Canary Wharf, North Greenwich, ...",38,15
2,"[New Cross Gate, Stepney Green]","[New Cross Gate, Surrey Quays, Canada Water, R...",26,14
3,"[Ealing Broadway, South Kensington]","[Ealing Broadway, Ealing Common, Acton Town, T...",49,20
4,"[Mile End, Baker Street]","[Mile End, Bethnal Green, Liverpool Street, Ba...",84,20
5,"[Baker Street, Wembley Park]","[Baker Street, Finchley Road, Wembley Park]",15,13


In [12]:
mean_bfs = np.mean(df['Exploration Number'])
mean_bfsc = np.mean(df['Cost in minutes'])
print(mean_bfs,mean_bfsc)

42.0 14.833333333333334


In [13]:
data_dfs = [[], [], [], [], [], []]

for i, x in enumerate(cities):
    p, n, c = dfs_implementation(t_graph, x[0], x[1], compute_exploration_cost=True)
    data_dfs[i].append(cities[i])
    data_dfs[i].append(p)
    data_dfs[i].append(n)
    data_dfs[i].append(c)

In [14]:
# Data for dfs
df1 = pd.DataFrame(data_dfs, columns = col_names)
df1

,Train stations,Route,Exploration Number,Cost in minutes
0,"[Euston, Victoria]","[Euston, Warren Street, Goodge Street, Tottenh...",136,15
1,"[Canada Water, Stratford]","[Canada Water, Canary Wharf, North Greenwich, ...",246,15
2,"[New Cross Gate, Stepney Green]","[New Cross Gate, Surrey Quays, Canada Water, C...",24,25
3,"[Ealing Broadway, South Kensington]","[Ealing Broadway, Ealing Common, North Ealing,...",45,58
4,"[Mile End, Baker Street]","[Mile End, Bow Road, Bromley-by-Bow, West Ham,...",41,74
5,"[Baker Street, Wembley Park]","[Baker Street, Finchley Road, Wembley Park]",9,13


In [15]:
mean_dfs = np.mean(df1['Exploration Number'])
mean_dfsc = np.mean(df1['Cost in minutes'])
print(mean_dfs,mean_dfsc)

83.5 33.333333333333336


In [16]:
data_ucs = [[], [], [], [], [], []]

for i, x in enumerate(cities):
    p, n, c = ucs_implementation(t_graph, x[0], x[1])
    data_ucs[i].append(cities[i])
    data_ucs[i].append(p)
    data_ucs[i].append(n)
    data_ucs[i].append(c)

In [17]:
df2 = pd.DataFrame(data_ucs, columns = col_names)
df2

,Train stations,Route,Exploration Number,Cost in minutes
0,"[Euston, Victoria]","[Euston, Warren Street, Oxford Circus, Green P...",35,7
1,"[Canada Water, Stratford]","[Canada Water, Rotherhithe, Wapping, Shadwell,...",59,14
2,"[New Cross Gate, Stepney Green]","[New Cross Gate, Surrey Quays, Canada Water, R...",20,14
3,"[Ealing Broadway, South Kensington]","[Ealing Broadway, Ealing Common, Acton Town, T...",47,20
4,"[Mile End, Baker Street]","[Mile End, Bethnal Green, Liverpool Street, Ba...",92,19
5,"[Baker Street, Wembley Park]","[Baker Street, Finchley Road, Wembley Park]",84,13


In [18]:
mean_ucs = np.mean(df2['Exploration Number'])
mean_ucsc = np.mean(df2['Cost in minutes'])
print(mean_ucs,mean_ucsc)

56.166666666666664 14.5


#### Reverse Method

In [19]:
# BFS in reverse
data_br = [[], [], [], [], [], []]

for i, x in enumerate(cities):
    p, n, c = bfs_implementation(station_dict1, x[0], x[1], compute_exploration_cost=True, reverse = True)
    data_br[i].append(cities[i])
    data_br[i].append(p)
    data_br[i].append(n)
    data_br[i].append(c)

In [20]:
df_br = pd.DataFrame(data_br, columns = col_names)
df_br

,Train stations,Route,Exploration Number,Cost in minutes
0,"[Euston, Victoria]","[Euston, Warren Street, Oxford Circus, Green P...",34,7
1,"[Canada Water, Stratford]","[Canada Water, Canary Wharf, North Greenwich, ...",25,15
2,"[New Cross Gate, Stepney Green]","[New Cross Gate, Surrey Quays, Canada Water, R...",40,14
3,"[Ealing Broadway, South Kensington]","[Ealing Broadway, Ealing Common, Acton Town, T...",47,20
4,"[Mile End, Baker Street]","[Mile End, Bethnal Green, Liverpool Street, Ba...",98,20
5,"[Baker Street, Wembley Park]","[Baker Street, Finchley Road, Wembley Park]",9,13


In [21]:
# DFS1 in reverse
data_d = [[], [], [], [], [], []]

for i, x in enumerate(cities):
    p, n, c = dfs_implementation(t_graph, x[0], x[1], compute_exploration_cost=True, reverse = True)
    data_d[i].append(cities[i])
    data_d[i].append(p)
    data_d[i].append(n)
    data_d[i].append(c)

In [22]:
df_d = pd.DataFrame(data_d, columns = col_names)
df_d

,Train stations,Route,Exploration Number,Cost in minutes
0,"[Euston, Victoria]","[Euston, King's Cross St. Pancras, Euston Squa...",52,62
1,"[Canada Water, Stratford]","[Canada Water, Rotherhithe, Wapping, Shadwell,...",224,20
2,"[New Cross Gate, Stepney Green]","[New Cross Gate, Surrey Quays, Canada Water, R...",267,14
3,"[Ealing Broadway, South Kensington]","[Ealing Broadway, West Acton, North Acton, Eas...",69,57
4,"[Mile End, Baker Street]","[Mile End, Bethnal Green, Liverpool Street, Ba...",11,22
5,"[Baker Street, Wembley Park]","[Baker Street, Edgware Road, Paddington, Baysw...",228,63


#### Task 2.3

In [23]:
def improved_ucs(graph, start_st, end_st):
    
    if start_st == end_st:
        return None
    
    number_of_explored_nodes = 0
    cost = 0
    explored = {}
    explored[start_st] = (cost, start_st)
    cost_to_change = 2
    line_list = []

    # Create priority queue
    p_queue = PriorityQueue()
    p_queue.put((0, [start_st]))
    
    while not p_queue.empty():
    
        cost, nodes = p_queue.get() 
    
        node = nodes[-1]
        number_of_explored_nodes += 1
        
        if len(nodes) > 1:
            parent_line_name = graph[nodes[-1]][nodes[-2]]['line']
            line_list.append(parent_line_name)
            
        
        if node == end_st:
            print('number of explorations = {}'.format(number_of_explored_nodes))
            print("Path is {}".format(explored[end_st][1]))
            print("The cost in average time is {} minutes".format(explored[end_st][0]))      
        
        neighbours = graph.neighbors(node)

        for neighbour in neighbours:
            # Get the cost of each node
            edge_weight = graph[node][neighbour]
            if len(edge_weight) != 0:
                cost_to_neighbour = edge_weight["weight"]
            else:
                cost_to_neighbour = 1
            line_name = edge_weight['line']
            
            new_cost = cost + cost_to_neighbour
            
            if len(line_list) > 0:
                if line_list[-1] != line_name:
                    new_cost = cost + cost_to_neighbour + cost_to_change
            
            if (neighbour not in explored) or (explored[neighbour][0]>new_cost):
                next_node = (new_cost, nodes+[neighbour])
                explored[neighbour] = next_node # Update the node with best value
                p_queue.put(next_node) # Also will add it as a possible path to explore
            
    
    return None

In [24]:
improved_ucs(t_graph, 'Baker Street', 'Wembley Park')

number of explorations = 64
Path is ['Baker Street', 'Finchley Road', 'Wembley Park']
The cost in average time is 13 minutes


In [25]:
improved_ucs(t_graph, 'Canada Water', 'Stratford')

number of explorations = 45
Path is ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
The cost in average time is 15 minutes


#### Task 2.4

In [26]:
def get_index(zone_d, station_name):
    zone_list = ['1', '2', '3', '4', '5', '6', 'a', 'b', 'c', 'd']
    for x in zone_d[station_name]:
        zone_index = zone_list.index(x)
    return zone_index 

In [27]:
# To implement Greedy BFS
def greedy_bfs(graph, start_st, end_st, zone_d):
    
    if start_st == end_st:
        return None
    
    number_of_explored_nodes = 0
    cost = 0
    explored = {}
    explored[start_st] = (cost, start_st)
    line_list = []

    # Create priority queue
    p_queue = PriorityQueue()
    p_queue.put((0, [start_st]))
    ind_goal = get_index(zone_d,end_st)
    
    while p_queue.empty() == False:
    
        ind, nodes = p_queue.get()
    
        node = nodes[-1]
        number_of_explored_nodes += 1
        
        if node == end_st:
            print('number of explorations = {}'.format(number_of_explored_nodes))
            path = explored[end_st][1]
            for i in range(len(path) - 1):
                cost += graph[path[i]][path[i + 1]]["weight"]
            print('Cost in average time = {} minutes'.format(cost))
            print("Path is {}".format(path))
        
        neighbours = graph.neighbors(node)

        for neighbour in neighbours:
            ind_neighbour = get_index(zone_d, neighbour)
            
            ind_difference = abs(ind_neighbour - ind_goal)
            new_cost = ind_difference
        
            
            if (neighbour not in explored) or (explored[neighbour][0]>new_cost):
                next_node = (new_cost, nodes+[neighbour])
                explored[neighbour] = next_node
                p_queue.put(next_node)
            
    
    return None    

In [28]:
greedy_bfs(t_graph, 'Baker Street', 'Wembley Park', zone_dict)

number of explorations = 3
Cost in average time = 13 minutes
Path is ['Baker Street', 'Finchley Road', 'Wembley Park']


In [29]:
greedy_bfs(t_graph, 'Canada Water', 'Stratford', zone_dict)

number of explorations = 10
Cost in average time = 15 minutes
Path is ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']


### Task3.1: Genetic Algorithm

In [30]:
from hints import closeness
import random
import numpy as np

In [31]:
def generateParent(length, n_iter, geneset):
    guess_list = []
    for n in range(n_iter):
        guess = list("")
        for i in range(0,length):
            geneIndex = random.randint(0, len(geneset) -1);
            guess.append(geneset[geneIndex])
        z = ''.join(guess)
        guess_list.append(z)
        
    return(guess_list)

def top_percent(percentage, n_iter, f_list):
    num_to_pick = int(percentage * n_iter)
    sorted_list = sorted(f_list.items(), key=lambda x:x[1], reverse=True)
    z = []
    for x, y in sorted_list:
        z.append(x)
    top_half = z[0:num_to_pick]
    top_two = z[0:2]
    
    return top_half, top_two

def crossover(p1, p2):
    c1, c2 = p1.copy(), p2.copy()
    split_pt = random.randint(1, len(p1)-2)
    c1 = p1[:split_pt] + p2[split_pt:]
    c2 = p2[:split_pt] + p1[split_pt:]
    return [c1, c2]

def get_crossover(percentage, n_iter, top_half):
    crossover_list = []
    num_to_crossover = int((n_iter - 2)/2)
    num_to_pick = int(percentage * n_iter)
    for n in range(num_to_crossover):
        num1_index = random.randint(0, int(num_to_pick -1))
        num2_index = random.randint(0, int(num_to_pick -1))
    
        p1 = list(top_half[num1_index])
        p2 = list(top_half[num2_index])
    
        x1, x2 = crossover(p1, p2)
        crossover_list.append(''.join(x1))
        crossover_list.append(''.join(x2))
    
    return crossover_list

def mutate(parent):
    total_Index = random.randint(0, len(total_gene) -1)
    index = random.randint(0, len(parent) - 1)
    list_p = list(parent)
    list_p[index] = total_gene[total_Index] 
    return(''.join(list_p))

In [32]:
random.seed(42)

In [33]:
def getBest(length, percent, n_iter, geneset):
    num_of_iter = 0
    parent_list = generateParent(length, n_iter, geneset) # n_iter is number of parent values you want to generate
    bestFitness = closeness(parent_list, "ec22181")
    
    while max(bestFitness.values()) < 1:
        num_of_iter += 1
        top_half1, top_two1 = top_percent(percent, n_iter, bestFitness)
        crossover_list = get_crossover(percent, n_iter, top_half1)
        
        child = []
        for x in crossover_list:
            child.append(mutate(x))
    
        children = child + top_two1
        childFitness = closeness(children, "ec22181")
        
        populationFitness = {**bestFitness,**childFitness}
        
        sorted_fitness = sorted(populationFitness.items(), key=lambda x:x[1], reverse=True)
        new_list = dict()
        for a in range(0, n_iter):
            for b, c in sorted_fitness:
                new_list[b] = c
        
        bestFitness = new_list   
         
    return bestFitness, num_of_iter

In [34]:
total_gene = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ_"
len_target = 10
best_fits, n = getBest(len_target, 0.5, 100, total_gene)
best_fit = list(best_fits)[0]
print('Number of iterations is {}, Secret phrase is {}'.format(n, best_fit))

Number of iterations is 37, Secret phrase is 6HJKXW3U7T


#### Task 3.3

In [ ]:
# Getting the mean and variance

n_val = []
for i in range(11):
    best_fits, n = getBest(len_target, 0.5, 100, total_gene)
    n_val.append(n)

mean_val = np.mean(n_val)
var_val = np.var(n_val)


print('Mean is {}, Variance is {}'.format(mean_val, var_val))

#### Task 3.4

In [ ]:
# Effect of changing number of population

pop_num = [10, 30, 50, 70, 90, 110, 130, 150]
n_val2 = {}
for i in pop_num:
    best_fits, n = getBest(len_target, 0.5, i, total_gene)
    n_val2[i] = n

print(n_val2)

In [ ]:
# Effect of changing number of percentage of selection

sel_num = [0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
n_val3 = {}
for j in sel_num:
    best_fits, n = getBest(len_target, j, 100, total_gene)
    n_val3[j] = n

print(n_val3)